In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 transformers datasets evaluate accelerate scikit-learn

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU Model: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU Model: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [3]:
dataset = load_dataset("dair-ai/emotion")

print("Example text:", dataset['train'][0]['text'])
print("Example label (int):", dataset['train'][0]['label'])

label_names = dataset['train'].features['label'].names
print(f"Available Labels: {label_names}")

Example text: i didnt feel humiliated
Example label (int): 0
Available Labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [4]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True, 
        max_length=512
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, 
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id
).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    
    fp16=True
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.260400,0.201638,0.928000
2,0.147700,0.184786,0.933000
3,0.097000,0.165340,0.938500


TrainOutput(global_step=3000, training_loss=0.25646459706624347, metrics={'train_runtime': 553.6842, 'train_samples_per_second': 86.692, 'train_steps_per_second': 5.418, 'total_flos': 6358888710144000.0, 'train_loss': 0.25646459706624347, 'epoch': 3.0})

In [9]:
def predict_emotion(text):
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        padding="max_length",
        truncation=True,
        max_length=512
    ).to(device)
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_class_id = logits.argmax().item()
    return model.config.id2label[predicted_class_id]

print("-" * 30)
print(f"Text: 'I passed my exam!' -> {predict_emotion('I passed my exam!')}")
print(f"Text: 'I am so happy today!' -> {predict_emotion('I am so happy today!')}")
print(f"Text: 'I am furious about this.' -> {predict_emotion('I am furious about this.')}")
print("-" * 30)

------------------------------
Text: 'I passed my exam!' -> joy
Text: 'I am so happy today!' -> joy
Text: 'I am furious about this.' -> anger
------------------------------


In [10]:
model.save_pretrained("my_emotion_model")
tokenizer.save_pretrained("my_emotion_model")

print("Model saved successfully!")

Model saved successfully!
